# Use Case and Model  Life cycle Governance with SageMaker Model Registry resource sharing

## ML Flow Experimentation with Shared Model Group

This notebook has been tested in Amazon SageMaker Studio with the SageMaker Distribution container 1.9 and Python 3 kernel.

In [ ]:
!pip install -r requirements.txt

### 1. Set-up

In [ ]:
import boto3
import mlflow
import pandas as pd
import os
import sagemaker
from time import gmtime, strftime
from sagemaker import get_execution_role
import json

bucket_name = sagemaker.Session().default_bucket()
prefix = "mlflow-credit-risk"

sagemaker_client = boto3.client("sagemaker")

s3_root_folder = f"s3://{bucket_name}/{prefix}"
sess = sagemaker.Session()

role = get_execution_role(sess)
print (f"Your Amazon SageMaker Execution role is: {role}")

**Access Model Package Groups in Shared Services account**

To be able to access Model Package Groups in a Shared Services AWS account, you'll need the following permissions assigned to the SageMaker execution role. 
Replace **\<YOUR_AWS_ACCOUNT\>** with your own AWS Account number. Replace **\<SHARED_SERVICES_ACCOUNT\>** with the Account number of the Shared Services account.
```json


    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "ram:GetResourceShareInvitations"
            ],
            "Resource": "arn:aws:ram:us-east-1:<YOUR_AWS_ACCOUNT>:resource-share-invitation/*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "ram:AcceptResourceShareInvitation"
            ],
            "Resource": "arn:aws:ram:us-east-1:<SHARED_SERVICES_ACCOUNT>:resource-share-invitation/*"
        }
    ]
}
```

Before you get started, check if there are any pending invitations from the shared services account 
and accept them. 
This will allow you to discover share model package groups and register your model versions against them.

In [ ]:
ram_client = boto3.client('ram')
response = ram_client.get_resource_share_invitations()
pending_invitations = []
# Review all pending invitations
for i in response['resourceShareInvitations']:
    if i['status'] == "PENDING":
        pending_invitations.append(i)
print(pending_invitations,sep='\n')

In [ ]:
# Accept the resource share invitation from shared services account
if pending_invitations:
    response = ram_client.accept_resource_share_invitation(resourceShareInvitationArn=pending_invitations[0]['resourceShareInvitationArn'])
    print(response)

To set up and manage an MLflow tracking server, as well as work with managed MLflow experiments, you'll need the following permissions assigned to the SageMaker execution role

```json
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Sid": "VisualEditor0",
			"Effect": "Allow",
			"Action": [
				"sagemaker:DeleteMlflowTrackingServer",
				"sagemaker:StartMlflowTrackingServer",
				"sagemaker:CreatePresignedMlflowTrackingServerUrl",
				"sagemaker:UpdateMlflowTrackingServer",
				"sagemaker:CreateMlflowTrackingServer",
				"sagemaker:StopMlflowTrackingServer"
			],
			"Resource": "*"
		},
		{
			"Sid": "VisualEditor1",
			"Effect": "Allow",
			"Action": [
				"sagemaker-mlflow:*"
			],
			"Resource": "*"
		}
	]
}
```

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = 'default'
if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

In [ ]:
def get_running_mlflow_server(sagemaker_client, status_filter=['Created', 'Creating']):
    for status in status_filter:
        servers = sagemaker_client.list_mlflow_tracking_servers(TrackingServerStatus=status)['TrackingServerSummaries']
        if servers:
            server = servers[0]
            print(f"Found an MLflow server {server['TrackingServerArn']} in the status '{status}'.")
            return server['TrackingServerArn'], server['TrackingServerName']
    print("No MLflow servers found.")
    return None, None

def create_mlflow_server(sagemaker_client, bucket_name, sm_role, domain_id):
    """
    Creates a new MLflow server and returns its ARN and name.
    """
    timestamp = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{domain_id}-{timestamp}"
    response = sagemaker_client.create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{timestamp}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = response['TrackingServerArn']
    print(f"Server creation request succeeded. The server {mlflow_arn} is being created.")
    return mlflow_arn, mlflow_name

# Get a running MLflow server or create a new one if none exists
mlflow_arn, mlflow_name = get_running_mlflow_server(sagemaker_client)
if not mlflow_arn:
    mlflow_arn, mlflow_name = create_mlflow_server(sagemaker_client, bucket_name, sm_role, domain_id)

### 2. Prepare the data

The code was adapted from this repository https://github.com/aws-samples/amazon-sagemaker-credit-risk-prediction-explainability-bias-detection/tree/main

In [ ]:
from sagemaker.s3 import S3Downloader
S3Downloader.download(
    "s3://sagemaker-sample-files/datasets/tabular/uci_statlog_german_credit_data/SouthGermanCredit.asc",
    "data",
)

In [ ]:
credit_columns = [
    "status",
    "duration",
    "credit_history",
    "purpose",
    "amount",
    "savings",
    "employment_duration",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "present_residence",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker",
    "credit_risk",
]

In [ ]:
training_data = pd.read_csv(
    "data/SouthGermanCredit.asc",
    names=credit_columns,
    header=0,
    sep=r" ",
    engine="python",
    na_values="?",
).dropna()

In [ ]:
test_data = training_data.sample(frac=0.1, random_state=42)
test_data = test_data.drop("credit_risk", axis=1)
test_columns = [
    "status",
    "duration",
    "credit_history",
    "purpose",
    "amount",
    "savings",
    "employment_duration",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "present_residence",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker",
]

training_data.to_csv("train.csv", index=False, header=True, columns=credit_columns)
test_data.to_csv("test.csv", index=False, header=True, columns=test_columns)

# save the datasets in S3 for future use
train_s3_url = sagemaker.Session().upload_data(
    path="train.csv",
    bucket=bucket_name,
    key_prefix=f"{prefix}/input"
)
print(f"Upload the dataset to {train_s3_url}")

test_s3_url = sagemaker.Session().upload_data(
    path="test.csv",
    bucket=bucket_name,
    key_prefix=f"{prefix}/input"
)
print(f"Upload the dataset to {test_s3_url}")


### 3. Process the data with Amazon SageMaker

In [ ]:
from time import gmtime, strftime, sleep

experiment_suffix = strftime('%d-%H-%M-%S', gmtime())
registered_model_name = f"credit-risk-model-{experiment_suffix}"
experiment_name = f"credit-risk-model-experiment-{experiment_suffix}"
print(experiment_name)

In [ ]:
import warnings
import pandas as pd
import numpy as np
import tarfile
import sklearn
import joblib
import mlflow
from sagemaker.s3 import S3Uploader
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer

from sklearn.exceptions import DataConversionWarning
from sagemaker.remote_function import remote


@remote(s3_root_uri=f"s3://{bucket_name}/{prefix}", dependencies=f"requirements.txt", instance_type="ml.m5.large")
def preprocess(df, experiment_name, mlflow_arn, bucket_name, prefix, run_id=None):
    """
    Preprocess the input data and split it into training and validation sets.

    Args:
        df (pandas.DataFrame): Input data.
        experiment_name (str): Name of the MLflow experiment.
        run_id (str, optional): MLflow run ID. If not provided, a new run will be created.
        mlflow_arn (str, optional): MLflow tracking URI.
        s3_root_folder (str, optional): S3 root folder for remote execution.

    Returns:
        tuple: A tuple containing the training and validation features and labels.
    """
    try:
        mlflow.set_tracking_uri(mlflow_arn)
        suffix = strftime('%d-%H-%M-%S', gmtime())
        mlflow.set_experiment(experiment_name=experiment_name if experiment_name else f"credit-risk-model-experiment-{suffix}")
        run = mlflow.start_run(run_id=run_id) if run_id else mlflow.start_run(run_name=f"remote-processing-{suffix}", nested=True)

        output_path = "/opt/ml/output/data"
        os.makedirs(output_path, exist_ok=True)

        print("Reading input data")
        model_dataset = mlflow.data.from_pandas(df)
        mlflow.log_input(model_dataset, context="model_dataset")

        print("Performing one-hot encoding")
        categorical_cols = [
            "credit_history",
            "purpose",
            "personal_status_sex",
            "other_debtors",
            "property",
            "other_installment_plans",
            "housing",
            "job",
            "telephone",
            "foreign_worker",
        ]
        transformer = make_column_transformer(
            (OneHotEncoder(sparse_output=False), categorical_cols),
            remainder="passthrough",
        )

        print("Preparing features and labels")
        X = df.drop("credit_risk", axis=1)
        y = df["credit_risk"]

        print("Building scikit-learn transformer")
        featurizer_model = transformer.fit(X)
        features = featurizer_model.transform(X)
        labels = LabelEncoder().fit_transform(y)

        split_ratio = 0.3
        print(f"Splitting data into train and validation sets with ratio {split_ratio}")
        X_train, X_val, y_train, y_val = train_test_split(
            features, labels, test_size=split_ratio, random_state=0
        )

        print(f"Train features shape after preprocessing: {X_train.shape}")
        print(f"Validation features shape after preprocessing: {X_val.shape}")

        mlflow.log_params({"train_shape": X_train.shape, "val_shape": X_val.shape})

        train_features_path = os.path.join(output_path, "train_features.csv")
        print(f"Saving training features to {train_features_path}")
        pd.DataFrame(X_train).to_csv(train_features_path, header=False, index=False)

        train_labels_path = os.path.join(output_path, "train_labels.csv")
        print(f"Saving training labels to {train_labels_path}")
        pd.DataFrame(y_train).to_csv(train_labels_path, header=False, index=False)

        val_features_path = os.path.join(output_path, "val_features.csv")
        print(f"Saving validation features to {val_features_path}")
        pd.DataFrame(X_val).to_csv(val_features_path, header=False, index=False)

        val_labels_path = os.path.join(output_path, "val_labels.csv")
        print(f"Saving validation labels to {val_labels_path}")
        pd.DataFrame(y_val).to_csv(val_labels_path, header=False, index=False)

        model_dir = "/opt/ml/model"
        os.makedirs(model_dir, exist_ok=True)
        model_path = os.path.join(model_dir, "model.joblib")
        model_output_path = os.path.join(model_dir, "model.tar.gz")

        print(f"Saving featurizer model to {model_output_path}")
        joblib.dump(featurizer_model, model_path)
        with tarfile.open(model_output_path, "w:gz") as tar:
            tar.add(model_path, arcname="model.joblib")

        mlflow.sklearn.log_model(
            sk_model=featurizer_model,
            artifact_path="processing/model",
            registered_model_name="sk-learn-model",
        )  
        return X_train, X_val, y_train, y_val
        
    except Exception as e:
        print(f"Exception in processing script: {e}")
        raise e
    finally:
        mlflow.end_run()

In [ ]:
df = pd.read_csv("train.csv", names=None, header=0, sep=",")
X_train, X_val, y_train, y_val = preprocess(df, experiment_name, mlflow_arn, bucket_name, prefix)

### 4. Model training with SageMaker training jobs

In [ ]:
import xgboost
from sklearn.metrics import roc_auc_score
import pickle as pkl
import os
import mlflow
import tarfile

@remote(s3_root_uri=f"s3://{bucket_name}/{prefix}", dependencies=f"requirements.txt", instance_type="ml.m5.large")
def train(X, val_X, y, val_y, num_round, params, mlflow_arn, experiment_name,run_id=None):
    output_path = "/opt/ml/model"
    mlflow.set_tracking_uri(mlflow_arn)
    mlflow.autolog()
    
    suffix = strftime('%d-%H-%M-%S', gmtime())
    mlflow.set_experiment(experiment_name=experiment_name if experiment_name else f"credit-risk-model-experiment-{suffix}")
    run = mlflow.start_run(run_id=run_id) if run_id else mlflow.start_run(run_name=f"remote-training-{suffix}", nested=True)

    try:
        os.makedirs(output_path, exist_ok=True)
        print(f"Directory '{output_path}' created successfully.")
    except OSError as e:
        print(f"Error creating directory '{output_path}': {e}")
        
    dtrain = xgboost.DMatrix(X, label=y)
    dval = xgboost.DMatrix(val_X, label=val_y)

    dtrain = xgboost.DMatrix(X, label=y)
    dval = xgboost.DMatrix(val_X, label=val_y)

    watchlist = [(dtrain, "train"), (dval, "validation")]
    mlflow.log_params(params)

    print("Training the model")
    evaluation__results = {}
    bst = xgboost.train(
        params=params, dtrain=dtrain, evals=watchlist, num_boost_round=num_round
    )
    pkl.dump(bst, open(output_path + "/model.bin", "wb"))

     # Compress the model.bin artifact to a tar file
    tar_filename = f"{output_path}/model.tar.gz"
    with tarfile.open(tar_filename, "w:gz") as tar:
        tar.add(f"{output_path}/model.bin", arcname="model.bin")

    # Upload the compressed model to S3
    # s3_client = boto3.client("s3")
    # s3_key = f"{s3_prefix}/model_{run.info.run_id}.tar.gz"
    # s3_client.upload_file(tar_filename, s3_bucket, s3_key)

    mlflow.log_artifact(local_path=tar_filename)


In [ ]:
hyperparameters = {
    "max_depth": "5",
    "eta": "0.1",
    "gamma": "4",
    "min_child_weight": "6",
    "silent": "1",
    "objective": "binary:logistic",
    "num_round": "100",
    "subsample": "0.8",
    "eval_metric": "auc"
}
num_round = 50

train(X_train, X_val, y_train, y_val,num_round, hyperparameters, mlflow_arn, experiment_name)

### 5. Register your the candidate model to the model registry in the shared services account

Now register the trained model in the MLflow model registry. The model is also automatically registered in the SageMaker model registry.

In [ ]:
from mlflow.entities import ViewType

run_filter = f"""
attributes.run_name LIKE "%training%"
attributes.status = 'FINISHED'
"""

runs_with_filter = mlflow.search_runs(
    experiment_names=[experiment_name],
    run_view_type=ViewType.ACTIVE_ONLY,
    filter_string=run_filter,
    order_by=["metrics.`validation-auc` DESC"],
)
best_run = runs_with_filter[:1]

In [ ]:
artifact_uri = best_run['artifact_uri'][0]

In [ ]:
response = sagemaker_client.list_model_package_groups(CrossAccountFilterOption="CrossAccount")
model_package_group_arn = response['ModelPackageGroupSummaryList'][0]['ModelPackageGroupArn']
print(model_package_group_arn)

In [ ]:
print(f"{artifact_uri}/model/model.tar.gz")

In [ ]:
import time

modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": "885854791233.dkr.ecr.us-east-1.amazonaws.com/sagemaker-distribution-prod@sha256:9e7622bbe2f3ee9dd516797bfe3ed310983b96190eeefbdeeeea69519d3946fe",
    	    "ModelDataUrl": f"{artifact_uri}/model.tar.gz"
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   },
    "ModelPackageGroupName" : model_package_group_arn,
    "ModelPackageDescription" : "Model to detect credit risk",
    "ModelApprovalStatus" : "PendingManualApproval"
}

model_package_group_name = "model-group-" + str(round(time.time()))

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to detect credit risk",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

create_model_package_response = sagemaker_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

In [ ]:
create_model_package_response = sagemaker_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))